In [33]:
from PIL import Image
import os
import random
import numpy as np
import utils
import cv2
import matplotlib.pyplot as plt

In [36]:
# Constants
REFERENCE_LOZENGE_SIZE = (331, 364)  # Our known good size for the reference image
REFERENCE_IMAGE_PATH = "data/imgs/raw/img_001.jpg"
output_dir = "data/imgs/synth"
os.makedirs(output_dir, exist_ok=True)

# Load and get dimensions of reference image
reference_img = cv2.imread(REFERENCE_IMAGE_PATH)
ref_height, ref_width = reference_img.shape[:2]
print(f"Reference image size: {ref_width}x{ref_height}")

# Calculate the reference ratio (how much of the reference image width the lozenge should occupy)
REFERENCE_WIDTH_RATIO = REFERENCE_LOZENGE_SIZE[0] / ref_width
print(f"Reference width ratio: {REFERENCE_WIDTH_RATIO:.3f}")

# Get list of lozenge variations
lozenge_dir = "data/brand_assets/processed/nectar_lozenge"
lozenge_files = [f for f in os.listdir(lozenge_dir) if f.endswith(('.png', '.PNG'))]
if not lozenge_files:
    raise ValueError("No lozenge images found in the specified directory")

# Get list of background images
background_files = [f for f in os.listdir("data/imgs/raw") if f.endswith('.jpg')][:]
padding = 20

for bg_file in background_files:
    background_path = os.path.join("data/imgs/raw", bg_file)
    background = cv2.imread(background_path)
    
    # Randomly select a lozenge for this iteration
    selected_lozenge_file = random.choice(lozenge_files)
    lozenge_path = os.path.join(lozenge_dir, selected_lozenge_file)
    lozenge = cv2.imread(lozenge_path, cv2.IMREAD_UNCHANGED)
    
    # Get background dimensions
    bg_height, bg_width = background.shape[:2]
    
    # Calculate the scaled size maintaining the same relative proportion as the reference image
    scaled_width = int(bg_width * REFERENCE_WIDTH_RATIO)
    aspect_ratio = REFERENCE_LOZENGE_SIZE[1] / REFERENCE_LOZENGE_SIZE[0]  # height/width
    scaled_height = int(scaled_width * aspect_ratio)
    
    # Scale lozenge for this specific background
    scaled_lozenge_dynamic = utils.scale_img(lozenge, (scaled_width, scaled_height))
    
    print(f"\nProcessing {bg_file}")
    print(f"Using lozenge: {selected_lozenge_file}")
    print(f"Background size: {bg_width}x{bg_height}")
    print(f"Scaled lozenge size: {scaled_lozenge_dynamic.shape}")

    # Get dimensions for placement
    loz_height, loz_width = scaled_lozenge_dynamic.shape[:2]

    # Skip images that are too small
    if bg_width < loz_width + 2*padding or bg_height < loz_height + 2*padding:
        print(f"Skipping {bg_file} - image too small for lozenge")
        continue

    regions = {
        'upper_left': {
            'x_range': (padding, min(bg_width//2 - loz_width - padding, bg_width - loz_width - padding)),
            'y_range': (padding, min(bg_height//2 - loz_height - padding, bg_height - loz_height - padding))
        },
        'center': {
            'x_range': (bg_width//4, min(3*bg_width//4 - loz_width - padding, bg_width - loz_width - padding)),
            'y_range': (bg_height//4, min(3*bg_height//4 - loz_height - padding, bg_height - loz_height - padding))
        },
        'lower_left': {
            'x_range': (padding, min(bg_width//2 - loz_width - padding, bg_width - loz_width - padding)),
            'y_range': (bg_height//2, min(bg_height - loz_height - padding, bg_height - loz_height - padding))
        }
    }

    valid_regions = {}
    for name, region in regions.items():
        x_start, x_end = region['x_range']
        y_start, y_end = region['y_range']
        
        if x_end > x_start and y_end > y_start:
            valid_regions[name] = region

    if not valid_regions:
        print(f"Skipping {bg_file} - no valid regions found")
        continue

    chosen_region = random.choice(list(valid_regions.keys()))
    region = valid_regions[chosen_region]

    x_offset = random.randint(*region['x_range'])
    y_offset = random.randint(*region['y_range'])

    print(f">> PLACING @{chosen_region} REGION, POSITION ({x_offset}, {y_offset})")

    # BLEND
    loz = scaled_lozenge_dynamic.astype(np.float32) / 255.0
    alpha_channel = loz[:, :, 3]
    rgb_loz = loz[:, :, :3]

    # Extract ROI with correct dimensions
    roi = background[y_offset:y_offset+loz_height, x_offset:x_offset+loz_width].astype(np.float32) / 255.0

    for c in range(3):
        roi[:, :, c] = roi[:, :, c] * (1 - alpha_channel) + rgb_loz[:, :, c] * alpha_channel

    roi = (roi * 255).astype(np.uint8)
    background[y_offset:y_offset+loz_height, x_offset:x_offset+loz_width] = roi

    # Save with original filename plus suffix
    output_filename = f"synth_{bg_file}"
    output_path = os.path.join(output_dir, output_filename)
    cv2.imwrite(output_path, background)

Reference image size: 1418x1417
Reference width ratio: 0.233

Processing img_158.jpg
Using lozenge: Artboard 95.png
Background size: 1080x1920
Scaled lozenge size: (277, 248, 4)
>> PLACING @lower_left REGION, POSITION (20, 1433)

Processing img_023.jpg
Using lozenge: Artboard 9_1.png
Background size: 1208x1208
Scaled lozenge size: (309, 272, 4)
>> PLACING @lower_left REGION, POSITION (260, 718)

Processing img_026.jpg
Using lozenge: Artboard 96.png
Background size: 1080x1080
Scaled lozenge size: (277, 248, 4)
>> PLACING @center REGION, POSITION (492, 277)

Processing img_202.jpg
Using lozenge: Artboard 8_1.png
Background size: 4500x4500
Scaled lozenge size: (1154, 1046, 4)
>> PLACING @lower_left REGION, POSITION (167, 2793)

Processing img_020.jpg
Using lozenge: Artboard 94.png
Background size: 1080x1080
Scaled lozenge size: (277, 248, 4)
>> PLACING @lower_left REGION, POSITION (70, 739)

Processing img_009.jpg
Using lozenge: Artboard 2_1.png
Background size: 300x600
Scaled lozenge siz